In [1]:
import sys
sys.path.append('/home/jiajunb/neural-dimension-reduction')

In [2]:
import os

import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.nn import functional as F
from src.models.distance_modeling import SurveyorDataSet, Surveyor, thesis_kl_div_add_mse_loss

import matplotlib.pyplot as plt

from sklearn.metrics import classification_report

import seaborn as sns; sns.set()

import copy

torch.manual_seed(0)

In [3]:
def far_func2(sorted_dist: torch.tensor, indices: torch.tensor):
    return sorted_dist[:, -80:], indices[:, -80:]
#     n, d = sorted_dist.shape
    
#     tmp1, tmp2 = list(), list()
#     for i in range(n):
#         idx = torch.randint(low=(d - 21), high=(d - 1), size=(1, 20))
#         tmp1.append(sorted_dist[i][idx])
#         tmp2.append(indices[i][idx])
#     return torch.cat(tmp1, dim=0), torch.cat(tmp2, dim=0)

def close_func2(sorted_dist: torch.tensor, indices: torch.tensor):
    return sorted_dist[:, 1:81], indices[:, 1:81]

train_dataset = SurveyorDataSet.from_df('/home/jiajunb/neural-dimension-reduction/data/train.csv', close_func2, far_func2)
val_dataset = SurveyorDataSet.from_df('/home/jiajunb/neural-dimension-reduction/data/dev.csv', close_func2, far_func2)

RuntimeError: [enforce fail at CPUAllocator.cpp:64] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 11520000000 bytes. Error code 12 (Cannot allocate memory)


In [ ]:
print(f'size of train_dataset: {len(train_dataset)}, size of val_dataset: {len(val_dataset)}')

print(f'training set positive: {(train_dataset.labels == 1).sum()} negative: {(train_dataset.labels == 0).sum()}')
print(f'validation set positive: {(val_dataset.labels == 1).sum()} negative: {(val_dataset.labels == 0).sum()}')

In [ ]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=1000, pin_memory=True)

In [ ]:
weight_decay = 1e-5
learning_rate = 1e-5
num_epoches = 3

In [ ]:
device = torch.device('cuda:1')
model = Surveyor()


no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(
        nd in n for nd in no_decay) and p.requires_grad], 'weight_decay': weight_decay},
    {'params': [p for n, p in model.named_parameters() if any(
        nd in n for nd in no_decay) and p.requires_grad], 'weight_decay': 0.0}
]

optimizer = torch.optim.AdamW(params=optimizer_grouped_parameters, lr=learning_rate)

model = model.to(device)

In [ ]:
def train_one_epoch(train_loader, model, optimizer, verbose):
    model = model.to(device)
    model.train()
    loss_sum = 0.
    for i, batch in enumerate(train_loader):
        x1, x2, labels, q = batch
        x1, x2, labels, q = x1.to(device), x2.to(device), labels.to(device), q.to(device)
        logits, p, out1, out2, loss = model(x1, x2, q, labels)
        model.zero_grad()  # reset gradient
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
        if verbose and i % 20 == 0:
            print(f'training loss: {loss_sum / (i + 1):.4f}')
    return loss_sum / len(train_loader)

def val_one_epoch(val_loader, model):
    model.eval()
    loss_fn1 = nn.CrossEntropyLoss()
    loss_fn2 = thesis_kl_div_add_mse_loss
    preds_list = list()
    labels_list = list()
    val_xentropy_loss = 0.
    val_thesis_loss = 0.
    with torch.no_grad():
        for i, batch in enumerate(val_loader):
            x1, x2, labels, q = batch
            x1, x2, q = x1.to(device), x2.to(device), q.to(device)
            logits, p, out1, out2 = model(x1, x2, q, labels=None)
            preds = torch.argmax(F.softmax(logits, dim=1), dim=1)
            preds_list.append(preds.cpu())
            labels_list.append(labels.cpu())
            labels = labels.to(device)
            val_xentropy_loss += loss_fn1(logits, labels).item()
            val_thesis_loss += loss_fn2(p, q).item()
    y_preds = torch.cat(preds_list)
    y_golds = torch.cat(labels_list)
    accuracy = float((y_preds == y_golds).sum().item()) / len(y_preds)
    return val_xentropy_loss / len(y_preds), val_thesis_loss / len(y_preds), accuracy, y_preds, y_golds

In [ ]:
def train_with_eval(train_loader, val_loader, model, optimizer, num_epoches, verbose):
    best_model = None
    best_avg_xentropy_loss, best_avg_thesis_loss, best_val_accuracy = float('inf'), float('inf'), 0. 
    for epoch_idx in range(1, num_epoches + 1):
        avg_loss = train_one_epoch(train_loader, model, optimizer, False)
        avg_xentropy_loss, avg_thesis_loss, val_accuracy, val_y_preds, val_y_golds = val_one_epoch(val_loader, model)
        if val_accuracy >  best_val_accuracy:
            best_avg_xentropy_loss, best_avg_thesis_loss, best_val_accuracy = avg_xentropy_loss, avg_thesis_loss, val_accuracy
            best_model = copy.deepcopy(model.cpu())
            print(f'epoch [{epoch_idx}]/[{num_epoches}] training loss: {avg_loss:.4f} '
                  f'val_cross_entropy_loss: {avg_xentropy_loss:.4f} '
                  f'val_thesis_loss: {avg_thesis_loss:.4f} '
                  f'val_accuracy: {val_accuracy:.4f} ')
            print(len(val_y_golds))
            print(classification_report(val_y_golds, val_y_preds))
    return best_avg_xentropy_loss, best_avg_thesis_loss, best_val_accuracy, best_model, model

In [ ]:
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=1000, pin_memory=True)

In [ ]:
best_avg_xentropy_loss, best_avg_thesis_loss, best_val_accuracy, best_model, final_model = train_with_eval(train_loader, val_loader, model, optimizer, num_epoches, True)


In [ ]:
best_avg_xentropy_loss, best_avg_thesis_loss, best_val_accuracy

In [ ]:
def far_func2(sorted_dist: torch.tensor, indices: torch.tensor):
    return sorted_dist[:, 21:], indices[:, 21:]

#     n, d = sorted_dist.shape
    
#     tmp1, tmp2 = list(), list()
#     for i in range(n):
#         idx = torch.randint(low=(d - 21), high=(d - 1), size=(1, 20))
#         tmp1.append(sorted_dist[i][idx])
#         tmp2.append(indices[i][idx])
#     return torch.cat(tmp1, dim=0), torch.cat(tmp2, dim=0)

def close_func2(sorted_dist: torch.tensor, indices: torch.tensor):
    return sorted_dist[:, 1:21], indices[:, 1:21]

test_dataset = SurveyorDataSet.from_df('/home/jiajunb/neural-dimension-reduction/data/processed/sample/dev.csv', close_func2, far_func2)

In [ ]:
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=1000, pin_memory=True)
avg_xentropy_loss, avg_thesis_loss, val_accuracy, val_y_preds, val_y_golds = val_one_epoch(test_loader, best_model.to(device))

print(f'val_cross_entropy_loss: {avg_xentropy_loss:.4f} '
      f'val_thesis_loss: {avg_thesis_loss:.4f} '
      f'val_accuracy: {val_accuracy:.4f} ')

In [ ]:
a = torch.tensor(val_y_preds == val_y_golds, dtype=torch.float32)

In [ ]:
bin_size = 500
acc_list = list()
for i in range(len(a) // bin_size):
    tmp = a[bin_size * i : bin_size * (i + 1)]
    acc_list.append(float(tmp.sum().item()) / len(tmp))

In [ ]:

plt.plot(range(len(acc_list)), acc_list)

In [ ]:
# torch.save({
#     "best_model": best_model.state_dict(),
#     "best_avg_xentropy_loss": best_avg_xentropy_loss,
#     "best_avg_thesis_loss": best_avg_thesis_loss, 
#     "best_val_accuracy": best_val_accuracy
# }, '../saves/surveyor.on.random_select_far.full.100')